In [7]:
import re
import os
import random
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import math
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from urllib.parse import urlparse
from sklearn import preprocessing
import tldextract
%matplotlib inline
import random
import gc
import xgboost as xgb
from sklearn import svm
from tensorflow import keras
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SpatialDropout1D
from sklearn import metrics

In [8]:
SIZE = 100
BATCH_SIZE = 16
EPOCHS = 100
SEED = 0

In [9]:
os.environ['PYTHONHASHSEED']=str(SEED)

random.seed(SEED)

np.random.seed(SEED)

tf.random.set_seed(SEED)

In [10]:
df=pd.read_csv("/Users/abhinavshinow/Documents/GitHub/Mal_URL/Data/mal_1.csv")

In [11]:
df['type']=df['type'].replace({'phishing':1,'benign':0,'defacement':2,'malware':3})

In [14]:
#Feature Extraction methods

#Checks for https in the string
def has_https(s):
    scheme = urlparse(s).scheme
    if(scheme==''):
        return 0
    else:
        return 1    

#returns the count of www in the url
def www(s):
    return s.count('www')

#returns the count of digits in the url
def digit(s):
    return sum(i.isnumeric() for i in s)

#returns the count of alphabets in the url
def alpha(s):
    return sum(i.isalpha() for i in s)

#returns the count of special character's in the url
def spec_char(s):
    return sum((not(i.isalpha()) and not(i.isnumeric())) for i in s)

def punctuation_count(s):
    list = ['.','!','#','$','%','&',',',':']

#returns 1 if the url contains a //
def double_slash(s):
    if(s.count('//')>0):
        return 1
    else:
        return 0

#returns the count of / in the url
def spec_char_1(s):
    return s.count('/')

#returns the count of ! in the url
def spec_char_2(s):
    return s.count('!')

#returns the count of . in the url
def spec_char_3(s):
    return s.count('.')

#returns the count of @ in the url    
def spec_char_4(s):
    return s.count('@')

#returns the count of % in the url
def spec_char_5(s):
    return s.count('%')

#returns the count of ? in the url
def spec_char_6(s):
    return s.count('?')

#returns the count of = in the url
def spec_char_7(s):
    return s.count('=')

#returns the count of + in the url
def spec_char_8(s):
    return s.count('+')

#returns the count of - in the url
def spec_char_9(s):
    return s.count('-')

#returns the count of & in the url
def spec_char_10(s):
    return s.count('&')
    
#returns the count of & in the url
def spec_char_11(s):
    return s.count('~')

#returns the length of the query if it contains a query
def query_length(s):
    return len(urlparse(s).query)

#returns the domain of the url
def dom_suffix(s):
    domains={'com':1,'edu':2,'org':3,'net':4,'onion':5,'in':6}
    s=tldextract.extract(s).suffix
    if s in domains.keys():
        return domains[s]
    else:
        return 0

#returns the length of the domain
def dom_length(s):
    return len(tldextract.extract(s).domain)

#returns the length of the sub_domain
def subdom_length(s):
    return len(tldextract.extract(s).subdomain)

#returns the length of the url path
def path_length(s):
    return len(urlparse(s).path)

#returns 1 if url contains a query
def has_query(s):
    if len(urlparse(s).query)==0:
        return 0
    else:
        return 1
#Check's for an ip within the url
def ip(s):
    has_ip=re.search('(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
    '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
    '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
    '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', s)
    if has_ip:
        return 1
    else:
        return 0
#Checks for extensions within the url
def extension(s):
    if '.htm' in s:
        return 1
    elif '.html' in s:
        return 2
    elif '.js' in s:
        return 3
    elif '.css' in s:
        return 4
    elif '.exe' in s:
        return 5
    elif '.php' in s:
        return 6
    elif '.py' in s:
        return 7
    else:
        return 0

In [15]:
df['len']=df['url'].apply(lambda s : len(str(s)))

df['https'] = df['url'].apply(lambda s : has_https(s))

df['www'] = df['url'].apply(lambda s : www(s))

df['digit_count'] = df['url'].apply(lambda s: digit(s))

df['alpha_count'] = df['url'].apply(lambda s: alpha(s))

df['spec_char_count'] = df['url'].apply(lambda s: spec_char(s))

df['puntuation_count'] = df['url'].apply(lambda s: punctuation_count(s))

df['double_slash'] = df['url'].apply(lambda s: double_slash(s))

df['/'] = df['url'].apply(lambda s: spec_char_1(s))

df['!'] = df['url'].apply(lambda s: spec_char_2(s))

df['.'] = df['url'].apply(lambda s: spec_char_3(s))

df['@'] = df['url'].apply(lambda s: spec_char_4(s))

df['%'] = df['url'].apply(lambda s: spec_char_5(s))

df['?'] = df['url'].apply(lambda s: spec_char_6(s))

df['='] = df['url'].apply(lambda s: spec_char_7(s))

df['+'] = df['url'].apply(lambda s: spec_char_8(s))

df['-'] = df['url'].apply(lambda s: spec_char_9(s))

df['&'] = df['url'].apply(lambda s: spec_char_10(s))

df['~'] = df['url'].apply(lambda s: spec_char_11(s))

df['dom_suffix'] = df['url'].apply(lambda s: dom_suffix(s))

df['dom_length'] = df['url'].apply(lambda s: dom_length(s))

df['subdom_length'] = df['url'].apply(lambda s: subdom_length(s))

df['path_length'] = df['url'].apply(lambda s: path_length(s))

df['query_length'] = df['url'].apply(lambda s: query_length(s))

df['ip'] = df['url'].apply(lambda s: ip(s))

df['extension'] = df['url'].apply(lambda s: extension(s))

df.drop('url',axis = 1, inplace = True)

In [16]:
#Normalising the feauture values to between 0 and 1
x=['len','https','www','digit_count','alpha_count','spec_char_count','puntuation_count','double_slash','/','!','.','@','%','?','=','+','-','&','~','dom_suffix','dom_length','subdom_length','path_length','query_length','extension','ip']
min_max_scaler = preprocessing.MinMaxScaler()
column_names_to_normalize = x
x = df[column_names_to_normalize].values
x_scaled = min_max_scaler.fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = df.index)
df[column_names_to_normalize] = df_temp

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [17]:
x=df[['len','https','www','digit_count','alpha_count','spec_char_count','puntuation_count','double_slash','/','!','.','@','%','?','=','+','-','&','~','dom_suffix','dom_length','subdom_length','path_length','query_length','extension','ip']]
y=df['type']

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,shuffle='True',stratify=y)

In [19]:
#Logistic Regression Classifier
model=LogisticRegression(solver='lbfgs',max_iter=1000)
model.fit(x_train,y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
model.score(x_test,y_test)